In [2]:
import os, sys, platform, socket
print("Host:", socket.gethostname())
print("Python:", sys.executable)
print("CUDA_VISIBLE_DEVICES:", os.getenv("CUDA_VISIBLE_DEVICES"))

Host: submit63.mit.edu
Python: /work/submit/tzaklama/ct_NNVMC/venvs/ct_nnvmc_venv/bin/python
CUDA_VISIBLE_DEVICES: 0


In [3]:
import os, socket, sys
print("Host:", socket.gethostname())      # MUST be c1234 (NOT submit66)
print("Python:", sys.executable)          # /work/.../ct_nnvmc_venv/bin/python
!ls /dev/nvidia*                           # should list /dev/nvidia0, etc.
!nvidia-smi -L

Host: submit63.mit.edu
Python: /work/submit/tzaklama/ct_NNVMC/venvs/ct_nnvmc_venv/bin/python
/dev/nvidia0  /dev/nvidia2  /dev/nvidiactl   /dev/nvidia-uvm-tools
/dev/nvidia1  /dev/nvidia3  /dev/nvidia-uvm

/dev/nvidia-caps:
nvidia-cap1  nvidia-cap2
GPU 0: NVIDIA GeForce GTX 1080 Ti (UUID: GPU-8cc96b65-f552-c8c5-ad52-03e6616b2931)


In [4]:
# or JAX
import jax; jax.devices()

[CudaDevice(id=0)]

In [5]:
# non_intTest.py
"""
Tests model for non-interacting 1d chain of fermions.

1d non-int chain is calculated analytically for given configuration and then used as target for model.
"""
import jax, jax.numpy as jnp

In [6]:
## Calculate exact coefficients for non-interacting 1d chain of fermions
import numpy as np
import math

def slater_coeffs_1d_sitebasis(L, N, bc=None):
    """Return (coeffs_complex, basis_masks) for the noninteracting ground state."""
    if bc is None:
        bc = 'pbc' if (N % 2 == 1) else 'apbc'
    phi = 0.0 if bc == 'pbc' else np.pi

    # choose occupied momenta
    if N % 2 == 0:
        ns = np.arange(-N//2, N//2)
    else:
        ns = np.arange(-(N//2), N//2 + 1)
    ks = (2*np.pi*ns + phi)/L

    # all N-particle bitmasks (ascending order of positions)
    from itertools import combinations
    masks, positions = [], []
    for occ in combinations(range(L), N):
        masks.append(sum(1<<i for i in occ))
        positions.append(np.array(occ, dtype=int))
    positions = np.stack(positions)                          # (H, N)

    # build orbital matrix Phi[a,x] = e^{ik_a x}/sqrt(L)
    Phi = np.exp(1j*np.outer(ks, np.arange(L))) / np.sqrt(L) # (N, L)

    # for each configuration, slice columns at positions and take det
    coeffs = np.empty(len(masks), dtype=np.complex128)
    for idx, pos in enumerate(positions):
        coeffs[idx] = np.linalg.det(Phi[:, pos]) / np.sqrt(math.factorial(N))
    return coeffs, np.array(masks, dtype=np.uint64)


In [7]:
## Calculate exact ground-state energy for non-interacting 1D chain of fermions for reference
%cd /work/submit/tzaklama/ct_NNVMC
import config as C
def tb_E0_1d(L, N, t, bc=None):
    """
    Exact ground-state energy for non-interacting 1D chain.
    bc: 'pbc' or 'apbc' (auto-choose if None: pbc for odd N, apbc for even N)
    """
    if bc is None:
        bc = 'pbc' if (N % 2 == 1) else 'apbc'
    phi = 0.0 if bc == 'pbc' else np.pi
    # choose the N integers centered around 0
    if N % 2 == 0:
        ns = np.arange(-N//2, N//2)                # even N
    else:
        ns = np.arange(-(N//2), N//2 + 1)          # odd N
    ks = (2*np.pi*ns + phi)/L
    eps = -2*t*np.cos(ks)
    return np.sum(np.sort(eps))  # already lowest; sort is harmless


E_exact = tb_E0_1d(L=C.N_SITES, N=C.N_PART, t=2*C.T_HOP, bc='pbc')
print("Exact TB E0 =", E_exact)

/work/submit/tzaklama/ct_NNVMC
Exact TB E0 = -4.82842712474619


In [8]:
"""
Latest version of the code to train a Lattice-TransFormer on synthetic data for non-interacting fermions in 1D.
"""
import jax, jax.numpy as jnp, optax
from flax.training import train_state
import config as C
T_LIST = [1.2, 1.0, 0.4, 0.6, 0.8]          # example
V_LIST = [0.0, 0.0, 0.0, 0.0, 0.0]          # non-interacting, so V=0
N_LIST = [2, 3, 5, 4, 4]          # particle numbers (can all differ)

assert len(T_LIST)==len(V_LIST)==len(N_LIST)
G = len(T_LIST)              # number of Hamiltonians

# ---------------------------------------------------------------------------
# 2)  Build concatenated training set
# ---------------------------------------------------------------------------
OCC_ALL, LAM_ALL, TARGET_ALL, GID_ALL = [], [], [], []
for gid,(t,v,npart) in enumerate(zip(T_LIST, V_LIST, N_LIST)):
    # basis for this particle number --------------------
    if C.LATTICE == "1d":
        from phys_system.lattice1D import enumerate_fock, mask_to_array
        basis = enumerate_fock(C.N_SITES, npart)
        occ   = jnp.array([mask_to_array(m, C.N_SITES) for m in basis],
                          dtype=jnp.int32)
    else:
        import phys_system.honeycomb as hc
        basis = hc.enumerate_fock(C.N_SITES, npart)
        occ   = jnp.array([hc.mask_to_array(m, C.N_SITES) for m in basis],
                          dtype=jnp.int32)

    # λ-vector extended to include N --------------------
    lam_vec = jnp.array([t, v, npart], dtype=jnp.float32)
    lam     = jnp.tile(lam_vec, (len(basis),1))

    
    # synthetic target coefficients replaced by exact non-interacting ones
    #key  = jax.random.PRNGKey(C.SEED + gid)
    #targ = jax.random.normal(key, (len(basis), 2))*0.1
    coeffs, masks = slater_coeffs_1d_sitebasis(L=C.N_SITES, N=npart, bc='pbc')
    # TARGET aligned to OCC (Re, Im)
    targ = jnp.stack([jnp.array(coeffs.real), jnp.array(coeffs.imag)], axis=1)

    gid_vec = jnp.full((len(basis),), gid, dtype=jnp.int32)

    OCC_ALL.append(occ)
    LAM_ALL.append(lam)
    TARGET_ALL.append(targ)
    GID_ALL.append(gid_vec)

# concatenate everything --------------------------------
OCC     = jnp.concatenate(OCC_ALL,    axis=0)
LAM     = jnp.concatenate(LAM_ALL,    axis=0)   # shape (B,3)
TARGET  = jnp.concatenate(TARGET_ALL, axis=0)
GIDS    = jnp.concatenate(GID_ALL,    axis=0)   # shape (B,)

print("Total training states:", OCC.shape[0], "| Hamiltonians:", G)

# ---------- model -----------------------------------------------------------
from networks.model import LatticeTransFormer
model = LatticeTransFormer(n_sites=C.N_SITES, depth=8, d_model=256)

# ---------------------------------------------------------------------------
# 3)  Loss & training step
# ---------------------------------------------------------------------------
from Loss.loss import overlap_loss_multi, amp_phase_loss, loss_normDiff   
LOSS_TYPE = "overlap_multi"   # choose in config or set here

def create_state(rng):
    params = model.init(rng, OCC, LAM, train=False)
    tx     = optax.adam(1e-3)
    return train_state.TrainState.create(
        apply_fn=model.apply, params=params, tx=tx)

def loss_fn(params):
    preds = model.apply(params, OCC, LAM, train=False)
    if LOSS_TYPE == "overlap_multi":
        return overlap_loss_multi(preds, TARGET, GIDS, num_groups=G)
    elif LOSS_TYPE == "amp_phase":
        return amp_phase_loss(preds, TARGET, neighbours)
    elif LOSS_TYPE == "original":
        return loss_normDiff(preds, TARGET)
    else:
        raise ValueError("Unknown LOSS_TYPE")

@jax.jit
def train_step(state):
    loss, grads = jax.value_and_grad(loss_fn)(state.params)
    state = state.apply_gradients(grads=grads)
    return state, loss

# ---------- run -------------------------------------------------------------
print("JAX backend:", jax.default_backend().upper(),
      "| lattice:", C.LATTICE,
      f"| N={C.N_SITES}, Np={C.N_PART}")

state = create_state(jax.random.PRNGKey(42))
for epoch in range(3*C.EPOCHS):
    state, loss = train_step(state)
    if epoch % C.PRINT_EVERY == 0:
        print(f"epoch {epoch:4d}  loss = {float(loss):.4e}")

# ---------- output ----------------------------------------------------------
coeffs = model.apply(state.params, OCC, LAM, train=False)
print(f"\nFirst {C.N_PRINT} coefficients for (t,V)=({C.T_HOP},{0.0}):")
for i in range(min(C.N_PRINT, len(basis))):
    n_str = ''.join(str(int(b)) for b in OCC[i])
    re, im = map(float, coeffs[i])
    print(f"|{n_str}⟩ → {re:+.5f} {im:+.5f} i")


Total training states: 280 | Hamiltonians: 5
JAX backend: GPU | lattice: 1d | N=8, Np=4
epoch    0  loss = 5.4999e-01
epoch  300  loss = 4.0871e-03
epoch  600  loss = 1.5569e-05
epoch  900  loss = 6.7625e-04
epoch 1200  loss = 1.2515e-04
epoch 1500  loss = 3.3379e-07
epoch 1800  loss = 9.4113e-02
epoch 2100  loss = 2.4992e-04
epoch 2400  loss = 2.8610e-06
epoch 2700  loss = 2.6502e-01
epoch 3000  loss = 1.0524e-01
epoch 3300  loss = 7.5177e-03

First 10 coefficients for (t,V)=(0.5,0.0):
|11000000⟩ → -2.09092 +0.13990 i
|10100000⟩ → -3.42089 +1.80702 i
|10010000⟩ → -3.29993 +3.88999 i
|10001000⟩ → -1.73407 +5.24778 i
|10000100⟩ → +0.37670 +5.09048 i
|10000010⟩ → +1.72532 +3.50834 i
|10000001⟩ → +1.58021 +1.36985 i
|01100000⟩ → -1.10907 +1.72096 i
|01010000⟩ → -1.21666 +3.70218 i
|01001000⟩ → +0.39346 +5.08952 i


In [9]:
# Build the Hamiltonian for the non-interacting 1D chain of fermions
Array = jax.Array
def H_builder(C) -> Array:
    import itertools 
    from scipy.sparse import lil_matrix, csr_matrix 
    Ns              = C.N_SITES          # number of sites of the 1D chain
    Nparticelle     = C.N_PART    # number of particles

    # basis vectors  
    klattice = 2*np.pi/Ns * np.arange(Ns)                                               # k lattice points  
    linking_table = {jr: [np.mod(jr-1, Ns), np.mod(jr+1, Ns)] for jr in range(Ns)}      # 1D linking table 
    # generate Hilbert space 
    def n2occ(n,Nsize):
        binr=np.binary_repr(n,width=Nsize) 
        return binr
    def generate_combinations(N,Np):
        """ Generate all combinations of Np particles from N available positions, returning them as binary numbers. """
        combinations = itertools.combinations(range(N), Np)
        results = []
        for comb in combinations:
            binary_num = 0  # Start with an empty binary number (all zeros)
            for i in comb:
                binary_num |= (1 << i)  # Set the bit at position i
            results.append(binary_num)  # Append the binary number to the results
        return results
    ##### Nparticelle number of particles specified previously >> N=2*Ns 
    combinations = generate_combinations(N=Ns,Np=Nparticelle)
    Nstates = len(combinations)
    states = np.arange(Nstates)
    state2ind = dict(zip(combinations,states))
    print("Number of states =",Nstates)  
    ###### hopping 
    def hop(string,j1,j2): # hopping 
        Ncount = sum( int(string[j]) for j in range(j2) ) + sum( int(string[j]) for j in range(j1) )  
        if j2 >= j1: 
            hop_sign = Ncount
        else: 
            hop_sign = 1 + Ncount 
        # distruggo in j2 
        tmp = string 
        tmp = "o"+tmp+"o" 
        tmp = tmp[:1+j2]+"0"+tmp[2+j2:] 
        tmp_p = tmp[1:-1] 
        # creo in j1 
        tmp_p = "o"+tmp_p+"o" 
        tmp_p = tmp_p[:1+j1]+"1"+tmp_p[2+j1:]  
        out = tmp_p[1:-1]
        return out, (-1)**hop_sign
    # Build the Hamiltonian
    Htunnel = lil_matrix((Nstates, Nstates), dtype=np.float64)   # LIL format for easy assignment
    #Hint    = lil_matrix((Nstates, Nstates), dtype=np.float64)   # LIL format for easy assignment 
    #Hloc    = lil_matrix((Nstates, Nstates), dtype=np.float64)   # LIL format for easy assignment
    for key, index in state2ind.items():
        string = n2occ(n=key,Nsize=Ns)  
        occ_vals = np.array([int(bit) for bit in string])  
        posAH = np.where(occ_vals == 1)[0]
        #posAH_odd = posAH[posAH % 2 == 1]
        #Hloc[index,index] = len(posAH_odd)  # number of odd particles in the state 
        for xx in posAH: 
            jxx = linking_table[xx]
            #Pxx = [ occ_vals[j] for j in jxx]
            # print( xx , f'linking table, {jxx}', f'Pxx {Pxx}' , string , Pxx.count(1) )
            #Hint[index,index] += Pxx.count(1)/2 
            for yy in jxx:  
                if occ_vals[yy] == 0: 
                    string_new, segno = hop(string=string,j1=yy,j2=xx)
                    # print( xx , yy , f'new string {string_new}, sign {segno}' )
                    occ_new = np.array([int(bit) for bit in string_new])  
                    key_new = int(string_new,2) 
                    index_new = state2ind[key_new] 
                    Htunnel[index_new,index] += -segno
    # Convert LIL matrix to dense numpy array, then to jax array
    H_dense = np.array(Htunnel.todense(), dtype=np.float32)
    H_jax = jnp.array(H_dense)
    return H_jax

In [9]:
# sampler_jax.py
"""
Metropolis–Hastings sampler for spinless fermion Fock states at fixed N.
Targets p(σ) ∝ |ψ(σ; λ)|^2 using your trained JAX model.

API:
  occ_batch = sample_occ_batch(
      model_apply, params, lam_vec, L, N,
      num_samples=4096, burn_in=1024, thin=4, n_chains=16, rng_seed=0)

where
  • model_apply(params, occ, lam, train=False) -> (B, 2) [Re, Im]
  • lam_vec is a 1D array, e.g. [t, V, N]  or  [t, V]
  • returns occ_batch as int32 array of shape (num_samples, L)
"""

from typing import Callable, Tuple
import numpy as np
import jax
import jax.numpy as jnp


# ------------------------- utilities ----------------------------------------
def _random_occ(L: int, N: int, rng: np.random.Generator) -> np.ndarray:
    """Random bitstring with exactly N ones among L sites."""
    occ = np.zeros(L, dtype=np.int8)
    occ[rng.choice(L, size=N, replace=False)] = 1
    return occ

def _propose_exchange(occ: np.ndarray, rng: np.random.Generator) -> Tuple[np.ndarray, int, int]:
    """
    Propose a number-conserving move: pick one occupied site i and one empty site j and swap.
    Returns (new_occ, i, j).  If all 0s or 1s (should not happen), returns copy.
    """
    ones  = np.flatnonzero(occ == 1)
    zeros = np.flatnonzero(occ == 0)
    if len(ones) == 0 or len(zeros) == 0:
        return occ.copy(), -1, -1
    i = int(rng.choice(ones))
    j = int(rng.choice(zeros))
    new_occ = occ.copy()
    new_occ[i] = 0
    new_occ[j] = 1
    return new_occ, i, j

@jax.jit
def _coeffs_to_probs(coeff_ri: jnp.ndarray) -> jnp.ndarray:
    """(B,2)->(B,) probabilities |ψ|^2 with small epsilon for stability."""
    psi = coeff_ri[:, 0] + 1j * coeff_ri[:, 1]
    return (jnp.abs(psi) ** 2 + 1e-30).real

def _eval_probs(model_apply, params, occ_batch: np.ndarray, lam_vec: jnp.ndarray) -> np.ndarray:
    """Evaluate |ψ|^2 for a batch of integer 0/1 arrays (np) at fixed λ."""
    occ_j = jnp.asarray(occ_batch, dtype=jnp.int32)
    lam_b = jnp.tile(lam_vec[None, :], (occ_j.shape[0], 1))
    coeff = model_apply(params, occ_j, lam_b, train=False)        # (B,2)
    probs = _coeffs_to_probs(coeff)                               # (B,)
    # IMPORTANT: return a WRITEABLE NumPy array
    return np.array(probs, dtype=np.float64, copy=True)


# ------------------------- main sampler -------------------------------------
def sample_occ_batch(model_apply: Callable,
                     params,
                     lam_vec: jnp.ndarray,
                     L: int,
                     N: int,
                     num_samples: int = 4096,
                     burn_in: int = 1024,
                     thin: int = 4,
                     n_chains: int = 16,
                     rng_seed: int = 0) -> jnp.ndarray:
    """
    Return an array of shape (num_samples, L) with entries in {0,1},
    distributed approximately as |ψ(σ;λ)|^2, using n_chains independent MH chains.
    """
    assert 0 <= N <= L
    rng = np.random.default_rng(rng_seed)

    # allocate per-chain state
    chains = np.stack([_random_occ(L, N, rng) for _ in range(n_chains)], axis=0)  # (C, L)
    probs  = _eval_probs(model_apply, params, chains, lam_vec)                    # (C,)
    if not probs.flags.writeable:
        probs = probs.copy()

    # how many samples per chain (ceil)
    per_chain = (num_samples + n_chains - 1) // n_chains
    samples = []

    total_steps = burn_in + thin * per_chain
    for step in range(total_steps):
        # Propose for all chains in parallel (Python level)
        props = []
        for c in range(n_chains):
            occ_new, _, _ = _propose_exchange(chains[c], rng)
            props.append(occ_new)
        props = np.stack(props, axis=0)                                # (C, L)

        probs_new = _eval_probs(model_apply, params, props, lam_vec)   # (C,)
        # MH accept
        accept_ratio = probs_new / (probs + 1e-300)
        accept = rng.random(n_chains) < np.minimum(1.0, accept_ratio)

        # update chains & probs
        chains[accept] = props[accept]
        probs[accept]  = probs_new[accept]

        # record (after burn-in) with thinning
        if step >= burn_in and ((step - burn_in) % thin == 0):
            samples.append(chains.copy())

    # collect & trim
    if len(samples) == 0:
        # edge case: too small total_steps
        return jnp.asarray(chains[:num_samples], dtype=jnp.int32)

    samples = np.concatenate(samples, axis=0)           # (~per_chain, C, L)
    samples = samples.reshape(-1, L)                    # (C*~, L)
    samples = samples[:num_samples]                     # exact M
    return jnp.asarray(samples, dtype=jnp.int32)

In [10]:
## Given ground state wavefunction, what is ground state energy?
import observables.energy as E
import sampler.sampler_jax as sampler
# (Assumes you have (i) a sampler that draws σ ~ |ψ|^2 and (ii) H for the basis.)

model_apply = lambda params, occ, lam, train=False: \
    model.apply(params, occ, lam, train=train)

L, N, t = C.N_SITES, C.N_PART, C.T_HOP

# 1) Basis + index map (must match H ordering!) (For ED and beyond)
basis_masks = enumerate_fock(L, N)
occ_basis   = jnp.array([mask_to_array(m, L) for m in basis_masks], dtype=jnp.int32)
state_index = E.build_state_index([int(m) for m in basis_masks])

# 2) Hamiltonian for appropriate basis
H = H_builder(C) # can change this but currently C is set to npart = 4 and L = 8

Number of states = 70


In [ ]:
# sampler/sampler_jax.py  (pure JAX MH sampler; fixed-N occupations)
from typing import Callable, Tuple
import jax
import jax.numpy as jnp
from jax import lax
from functools import partial

EPS = 1e-30

def _coeffs_to_probs(coeff_ri: jnp.ndarray) -> jnp.ndarray:
    """(B,2)->(B,) probabilities |ψ|^2 (float32), numerically safe."""
    psi = coeff_ri[..., 0] + 1j * coeff_ri[..., 1]
    return (jnp.abs(psi)**2 + EPS).real.astype(jnp.float32)

def _eval_probs(model_apply: Callable, params, occ_batch: jnp.ndarray, lam_vec: jnp.ndarray) -> jnp.ndarray:
    """
    Evaluate |ψ|^2 for a batch of 0/1 int32 arrays at fixed λ.
    occ_batch: (C, L) int32, lam_vec: (K,) float32
    returns: (C,) float32
    """
    C = occ_batch.shape[0]
    lam_b = jnp.broadcast_to(lam_vec, (C, lam_vec.shape[-1]))
    coeff = model_apply(params, occ_batch, lam_b, train=False)  # (C, 2), JAX op
    return _coeffs_to_probs(coeff)

def _forward(model, params, occ_batch, lam_batch, *, train: bool):
    # shapes:
    #   occ_batch: (B, L) int32 in {0,1}
    #   lam_batch: (B, K) float32
    return model.apply(params, occ_batch, lam_batch, train=train)  # -> (B,2)

# 2) make a jitted, batched apply with 'train' as a static arg so it compiles once
def make_apply_batched(model):
    f = partial(_forward, model)
    return jax.jit(f, static_argnames=('train',))

apply_batched = make_apply_batched(model)

@partial(jax.jit, static_argnames=('train',))
def eval_probs(params, occ_batch, lam_batch, *, train: bool):
    coeff = apply_batched(params, occ_batch, lam_batch, train=train)   # (B,2)
    return _coeffs_to_probs(coeff)  

def _random_occ(key: jax.Array, L: int, N: int) -> jnp.ndarray:
    """Random bitstring with exactly N ones among L using Gumbel-top-k."""
    g = jax.random.gumbel(key, (L,))
    # take top-N positions
    _, idx = lax.top_k(g, N)               # idx: (N,)
    occ = jnp.zeros((L,), dtype=jnp.int32)
    occ = occ.at[idx].set(1)
    return occ

def _propose_exchange(keys: Tuple[jax.Array, jax.Array], occ: jnp.ndarray) -> jnp.ndarray:
    """
    Propose a number-conserving move by choosing one occupied and one empty site
    uniformly at random (via Gumbel-max) and swapping them.
    keys: (key_occ, key_emp)
    occ: (L,) int32 in {0,1}
    """
    key_occ, key_emp = keys
    L = occ.shape[0]
    # sample indices uniformly among occupied / empty positions
    g1 = jax.random.gumbel(key_occ, (L,))
    g0 = jax.random.gumbel(key_emp, (L,))
    i = jnp.argmax(jnp.where(occ.astype(bool), g1, -jnp.inf))     # occupied index
    j = jnp.argmax(jnp.where(~occ.astype(bool), g0, -jnp.inf))    # empty index
    # swap bits
    new = occ
    new = new.at[i].set(0)
    new = new.at[j].set(1)
    return new

def sample_occ_batch(model_apply: Callable,
                     params,
                     lam_vec: jnp.ndarray,
                     L: int,
                     N: int,
                     num_samples: int = 4096,
                     burn_in: int = 1024,
                     thin: int = 4,
                     n_chains: int = 16,
                     rng_seed: int = 0) -> jnp.ndarray:
    """
    Draw (num_samples, L) configurations approximately from |ψ(σ;λ)|^2
    by running n_chains independent MH chains on the GPU.

    model_apply(params, occ, lam, train=False) -> (B,2) [Re, Im] must be pure JAX.
    lam_vec: (K,) float32, e.g. [t, V, N]
    """
    assert 0 <= N <= L

    # --- PRNG setup
    key = jax.random.PRNGKey(rng_seed)
    k_init, k_scan = jax.random.split(key)

    # --- initialize chains (C, L) with exactly N ones each (vectorized)
    init_keys = jax.random.split(k_init, n_chains)
    chains0 = jax.vmap(_random_occ, in_axes=(0, None, None))(init_keys, L, N)  # (C, L)

    # --- initial probabilities on device
    probs0 = _eval_probs(model_apply, params, chains0, lam_vec)               # (C,)

    # --- how many samples per chain to hit num_samples exactly
    per_chain = (num_samples + n_chains - 1) // n_chains
    total_steps = burn_in + thin * per_chain

    # pre-allocate save buffer
    saved0 = jnp.zeros((per_chain, n_chains, L), dtype=jnp.int32)
    save_idx0 = jnp.array(0, dtype=jnp.int32)

    """ def one_step(carry, step_idx):
        
        #Single MH step across all chains, all on-device.
        #Saves states after burn-in at thinning interval into a ring buffer.
        
        key, chains, probs, save_idx, saved = carry
        key, k1, k2, k3 = jax.random.split(key, 4)

        # propose for all chains in parallel (no Python loop)
        g1 = jax.random.gumbel(k1, (n_chains, L))
        g0 = jax.random.gumbel(k2, (n_chains, L))
        i_occ = jnp.argmax(jnp.where(chains == 1, g1, -jnp.inf), axis=1)  # (C,)
        i_emp = jnp.argmax(jnp.where(chains == 0, g0, -jnp.inf), axis=1)  # (C,)
        ar = jnp.arange(n_chains, dtype=jnp.int32)

        props = chains.at[ar, i_occ].set(0)
        props = props.at[ar, i_emp].set(1)

        probs_new = _eval_probs(model_apply, params, props, lam_vec)         # (C,)
        accept_ratio = probs_new / (probs + EPS)
        u = jax.random.uniform(k3, (n_chains,))
        accept = u < jnp.minimum(1.0, accept_ratio)

        chains_new = jnp.where(accept[:, None], props, chains)
        probs_new2 = jnp.where(accept, probs_new, probs)

        # save if we're past burn-in and on a thinning step
        keep = jnp.logical_and(step_idx >= burn_in,
                               jnp.equal(jnp.remainder(step_idx - burn_in, thin), 0))
        saved = lax.cond(
            keep,
            lambda buf: buf.at[save_idx].set(chains_new),
            lambda buf: buf,
            saved
        )
        save_idx = save_idx + keep.astype(jnp.int32)

        carry_out = (key, chains_new, probs_new2, save_idx, saved)
        return carry_out, None

    # --- run the MH scan
    carry0 = (k_scan, chains0, probs0, save_idx0, saved0)
    carry_out, _ = lax.scan(one_step, carry0, jnp.arange(total_steps))
    _, chains_f, _, _, saved = carry_out """

    # --- flatten samples and return exactly num_samples
    # saved: (per_chain, C, L) -> (per_chain*C, L)
    @partial(jax.jit, donate_argnums=(1,2,4))  # donate chains, probs, saved
    def run_mh_loop(key, chains0, probs0, saved0, save_idx0, lam_batch_C, total_steps, burn_in, thin):
        def one_step(carry, step_idx):
            key, chains, probs, save_idx, saved = carry
            key, k1, k2, k3 = jax.random.split(key, 4)
            C, L = chains.shape

            # propose via masked-gumbel argmax (same as before)
            g1 = jax.random.gumbel(k1, (C, L))
            g0 = jax.random.gumbel(k2, (C, L))
            i_occ = jnp.argmax(jnp.where(chains == 1, g1, -jnp.inf), axis=1)
            i_emp = jnp.argmax(jnp.where(chains == 0, g0, -jnp.inf), axis=1)
            ar = jnp.arange(C, dtype=jnp.int32)
            props = chains.at[ar, i_occ].set(0)
            props = props.at[ar, i_emp].set(1)

            # *** batched jitted model eval here ***
            probs_new = eval_probs(state.params, props, lam_batch_C, train=False)
            accept_ratio = probs_new / (probs + 1e-30)
            u = jax.random.uniform(k3, (C,))
            accept = u < jnp.minimum(1.0, accept_ratio)

            chains_new = jnp.where(accept[:, None], props, chains)
            probs_new2 = jnp.where(accept, probs_new, probs)

            keep = jnp.logical_and(step_idx >= burn_in,
                                (step_idx - burn_in) % thin == 0)
            saved = lax.cond(
                keep,
                lambda buf: buf.at[save_idx].set(chains_new),
                lambda buf: buf,
                saved
            )
            save_idx = save_idx + keep.astype(jnp.int32)
            return (key, chains_new, probs_new2, save_idx, saved), None

        (key_f, chains_f, probs_f, save_idx_f, saved_f), _ = lax.scan(
            one_step,
            (key, chains0, probs0, save_idx0, saved0),
            jnp.arange(total_steps),
        )
        return saved_f
    
    def make_lam_batch(lam_vec, B):
        lam_vec = jnp.asarray(lam_vec, dtype=jnp.float32)
        return jnp.broadcast_to(lam_vec, (B, lam_vec.shape[-1]))

    lam_batch_C = make_lam_batch(lam_vec, C)

    saved = run_mh_loop(key, chains0, probs0, saved0, save_idx0, lam_batch_C, total_steps, burn_in, thin)


    samples = saved.reshape(-1, L)[:num_samples]
    return samples.astype(jnp.int32)


: 

In [17]:
# 3) Draw M samples σ ~ |ψ|^2  (use your sampler; pseudo-code shown)
M = 1024
lam_vec   = jnp.array([2*t, 0.0, float(N)], dtype=jnp.float32) # non-interacting, so V=0 # C.T_HOP is 0.5
lam_batch = jnp.tile(lam_vec, (M, 1))

# 1) Draw configurations σ ~ |ψ|^2
occ_batch = sample_occ_batch(model_apply, state.params, lam_vec,
                             L=L, N=N,
                             num_samples=M, burn_in=256, thin=4,
                             n_chains=32, rng_seed=0)

def model_fn(occ, lam):
    coeff = model_apply(state.params, occ, lam, train=False)  # (B,2)
    return (coeff[:, 0] + 1j * coeff[:, 1]).astype(jnp.complex64)

# 4) Monte Carlo energy (module function averages local energies)
E_mc = E.expectation_local_energy(model=model_fn,
                      occ_batch=occ_batch,
                      params_batch=lam_batch,
                      H=H,
                      occ_basis=occ_basis,
                      state_index=state_index)
print("E0(model, MC) =", float(E_mc)) 

E0(model, MC) = -4.187276840209961


In [14]:
from functools import partial
jax.config.update("jax_enable_x64", False)
def psi_fn(params, occ, lam):
    # vectorizes over batch outside; single sample here
    c = model_apply(params, occ, lam, train=False)  # shape (2,)
    return (c[0] + 1j*c[1]).astype(jnp.complex64)

# Local energy for a **single** configuration; must be pure JAX
def local_energy_single(occ, lam, params, H, occ_basis, state_index):
    # --- IMPORTANT ---
    # * No Python loops
    # * No numpy/scipy
    # * H must be represented so its actions are JAX ops (or precomputed arrays)
    psi = psi_fn(params, occ, lam)
    # ... compute E_loc purely with jnp / lax ...
    # placeholder:
    return jnp.real(psi) * 0.0  # replace with your jax-only E_loc

# Batch it, then jit it. Mark python/structure args static so XLA doesn't recompile.
batched_local_energy = jax.jit(
    jax.vmap(local_energy_single, in_axes=(0,0,None,None,None,None)),
    static_argnames=('H','occ_basis','state_index')
)

# Warmup compile (important to separate compile vs run time):
e_loc_temp = batched_local_energy(occ_batch, lam_batch, state.params, H, occ_basis, state_index).block_until_ready()

ValueError: Non-hashable static arguments are not supported. An error occurred while trying to hash an object of type <class 'jaxlib._jax.ArrayImpl'>, [[ 0. -1.  0. ...  0.  0.  0.]
 [-1.  0. -1. ...  0.  0.  0.]
 [ 0. -1.  0. ...  0.  0.  0.]
 ...
 [ 0.  0.  0. ...  0. -1.  0.]
 [ 0.  0.  0. ... -1.  0. -1.]
 [ 0.  0.  0. ...  0. -1.  0.]]. The error was:
TypeError: unhashable type: 'jaxlib._jax.ArrayImpl'


In [12]:
# Larger M for better accuracy, takes far longer and is marginally better
M = 4096
lam_vec   = jnp.array([t, 0.0, float(N)], dtype=jnp.float32) # non-interacting, so V=0
lam_batch = jnp.tile(lam_vec, (M, 1))

# 1) Draw configurations σ ~ |ψ|^2
occ_batch = sample_occ_batch(model_apply, state.params, lam_vec,
                             L=L, N=N,
                             num_samples=M, burn_in=1024, thin=4,
                             n_chains=32, rng_seed=0)

def model_fn(occ, lam):
    coeff = model_apply(state.params, occ, lam, train=False)  # (B,2)
    return (coeff[:, 0] + 1j * coeff[:, 1]).astype(jnp.complex64)

# 4) Monte Carlo energy (module function averages local energies)
E_mc = E.expectation_local_energy(model=model_fn,
                      occ_batch=occ_batch,
                      params_batch=lam_batch,
                      H=H,
                      occ_basis=occ_basis,
                      state_index=state_index)
print("E0(model, MC) =", float(E_mc))

E0(model, MC) = -4.828277587890625


In [9]:
# use the SAME H you pass to expectation_local_energy (straight exact diagonalization)
evals = jnp.linalg.eigvalsh(H)      # H is (Hdim,Hdim) in N-sector
E_exact_from_H = float(evals[0])
print("E0(H) =", E_exact_from_H)

E0(H) = -4.828428745269775


In [8]:
# Rayleigh quotient method
# psi_full on the full basis used to build H
psi   = model_fn(occ, lam)
E_ray = float((jnp.vdot(psi, H @ psi) / (jnp.vdot(psi, psi) + 1e-12)).real) ## Hilbert space needs to be same for model and H
# Throws error because model is npart = 3 and L = 8 while H is npart = 4 and L = 8
print("E0(Rayleigh) =", E_ray)


E0(Rayleigh) = -3.919802665710449


In [46]:
E_mc = float(E.expectation_local_energy(
              model=model_fn, occ_batch=occ_batch,
              params_batch=lam_batch, H=H,
              occ_basis=occ_basis, state_index=state_index).real)
print("E0(model, MC) =", E_mc)

E0(model, MC) = -3.914226531982422


In [ ]:
# local_energy_1d.py  (drop-in helper)

"""

def local_energy_1d_tb(model_fn, occ_batch: jnp.ndarray, lam_batch: jnp.ndarray,
                       t: float, bc: str = "pbc") -> jnp.ndarray:
    M, L = occ_batch.shape
    N = jnp.sum(occ_batch[0])  # fixed-N sampler
    # base amplitudes
    psi = model_fn(occ_batch, lam_batch)  # (M,)
    eps = 1e-30

    # helper to flip i->j
    def move(occ_np, i, j):
        occ2 = occ_np.copy()
        occ2[i] = 0; occ2[j] = 1
        return occ2

    # JW sign for wrap hops:
    jw = 1.0 if (int(N) % 2 == 1) else -1.0
    bc_phase = 1.0 if bc == "pbc" else -1.0  # add a twist if you want APBC
    wrap_sign = jw * bc_phase

    E_loc = np.zeros(M, dtype=np.complex64)

    # loop in Python (clear & reliable; vectorise later if needed)
    for m in range(M):
        occ = np.asarray(occ_batch[m])
        lam = np.asarray(lam_batch[m])
        denom = complex(psi[m])
        if abs(denom) < eps:
            continue
        acc = 0.0 + 0.0j
        for i in range(L):
            if occ[i] == 0: 
                continue
            # hop right
            j = (i + 1) % L
            if occ[j] == 0:
                occ2 = move(occ, i, j)
                # wrap sign if i->j crosses boundary
                sgn = wrap_sign if (i == L-1 and j == 0) else 1.0
                num = complex(model_fn(jnp.asarray(occ2[None,:], dtype=jnp.int32),
                                       jnp.asarray(lam[None,:], dtype=jnp.float32))[0])
                acc += -t * sgn * (num / denom)
            # hop left
            j = (i - 1) % L
            if occ[j] == 0:
                occ2 = move(occ, i, j)
                sgn = wrap_sign if (i == 0 and j == L-1) else 1.0
                num = complex(model_fn(jnp.asarray(occ2[None,:], dtype=jnp.int32),
                                       jnp.asarray(lam[None,:], dtype=jnp.float32))[0])
                acc += -t * sgn * (num / denom)
        E_loc[m] = acc
    return jnp.asarray(E_loc)

E_loc = local_energy_1d_tb(model_fn, occ_batch, lam_batch, C.T_HOP, bc="pbc")
E_mc  = float(jnp.mean(E_loc).real)
print("E0(model, local energy) =", E_mc) """

E0(model, local energy) = -1.9571131467819214


In [ ]:
## Old version sketch of how to get local energy 
""" ## Given ground state wavefunction, what is ground state energy?
import observables.energy as E
import sampler.mcmc as sampler
# (Assumes you have (i) a sampler that draws σ ~ |ψ|^2 and (ii) H for the basis.)

model_apply = lambda params, occ, lam, train=False: \
    model.apply(params, occ, lam, train=train)

# 1) Basis + index map (must match H ordering!)
basis_masks = enumerate_fock(C.N_SITES, C.N_PART)
occ_basis   = jnp.array([mask_to_array(m, C.N_SITES) for m in basis_masks], dtype=jnp.int32)
state_index = E.build_state_index([int(m) for m in basis_masks])

# 2) Your Hamiltonian for that basis
# H = your_ed_builder(...)

# 3) Draw M samples σ ~ |ψ|^2  (use your sampler; pseudo-code shown)
M = 4096
lam_vec   = jnp.array([C.T_HOP, 0.0, C.N_PART], dtype=jnp.float32) # non-interacting, so V=0
lam_batch = jnp.tile(lam_vec, (M, 1))
# occ_batch = sampler.sample(model, state.params, lam_vec, M)  # shape (M, N_sites)
occ_batch = sampler.sample_chain(init_state: Array,
                 log_prob_fn: model_apply,
                 key: PRNGKey,
                 n_samples: 4096,
                 burn_in= 100,
                 thin=1)

# 4) Monte Carlo energy (module function averages local energies)
E_mc = E.expectation_local_energy(model=model_apply,
                      occ_batch=occ_batch,
                      params_batch=lam_batch,
                      H=H,
                      occ_basis=occ_basis,
                      state_index=state_index)
print("E0(model, MC) =", float(E_mc)) """

Number of states = 56
E0(H) = -4.828426837921143
